In [49]:
# importing libraries
import pandas as pd
import numpy as np
import ipywidgets
import difflib

In [50]:
first_name = input("Please insert the first name of the player you want to see trends for")
last_name = input("Please insert the last name of the player you want to see trends for")

In [51]:
full_name = first_name + " " + last_name
print(full_name)

Kevon Durent


In [52]:
# reading clean file
file_path = "C:/Github Repo/NBA-Analytics/data/basketballProject/cleaned-files/cleaned-data.csv"
clean_df = pd.read_csv(file_path)

In [53]:
fuzzy_df = pd.DataFrame([full_name],columns=["PLAYER"])
fuzzy_df["fuzzy_name"] = fuzzy_df["PLAYER"]
fuzzy_df["PLAYER"] = fuzzy_df["PLAYER"].apply(lambda x: difflib.get_close_matches(x, clean_df["PLAYER"])[0])

In [54]:
display(fuzzy_df)

,PLAYER,fuzzy_name
0,Kevin Durant,Kevon Durent


In [55]:
predicate_name = fuzzy_df["PLAYER"][fuzzy_df["fuzzy_name"] == full_name][0]

In [56]:
display(clean_df[clean_df["PLAYER"] == predicate_name])

,Unnamed: 0,index,Season #,PLAYER_ID,PLAYER,SEASON_ID,TEAM_ID,TEAM,Age,GP,...,TOV,PF,PTS,MOWAR,MDWAR,MPWAR,MIWAR,MWAR,MWARPM,MWARPG
415,415,415,0,201142,Kevin Durant,2007-08,1610612760,SEA,19.0,80,...,232,122,1624,2831.55,245.0,-120,2094.0,5050.55,1.824621,63.131875
416,416,416,1,201142,Kevin Durant,2008-09,1610612760,OKC,20.0,74,...,225,134,1871,3013.30,231.0,-60,2594.0,5778.30,2.002877,78.085135
417,417,417,2,201142,Kevin Durant,2009-10,1610612760,OKC,21.0,82,...,271,171,2472,3918.85,306.5,-120,3114.0,7219.35,2.228882,88.040854
418,418,418,3,201142,Kevin Durant,2010-11,1610612760,OKC,22.0,78,...,218,159,2161,3603.90,248.5,-12,2757.0,6597.40,2.171626,84.582051
419,419,419,4,201142,Kevin Durant,2011-12,1610612760,OKC,23.0,66,...,248,133,1850,2994.25,263.5,-51,2452.0,5658.75,2.222604,85.738636
420,420,420,5,201142,Kevin Durant,2012-13,1610612760,OKC,24.0,81,...,280,143,2280,3465.15,370.5,282,2987.0,7104.65,2.277861,87.711728
421,421,421,6,201142,Kevin Durant,2013-14,1610612760,OKC,25.0,81,...,285,174,2593,4107.30,237.0,480,2918.0,7742.30,2.479917,95.583951
422,422,422,7,201142,Kevin Durant,2014-15,1610612760,OKC,26.0,27,...,74,40,686,1113.50,78.0,108,799.0,2098.50,2.298467,77.722222
423,423,423,8,201142,Kevin Durant,2015-16,1610612760,OKC,27.0,72,...,250,137,2029,3297.40,239.5,333,2505.0,6374.90,2.472808,88.540278
424,424,424,9,201142,Kevin Durant,2016-17,1610612744,GSW,28.0,62,...,138,117,1555,2435.70,271.5,486,2019.0,5212.20,2.517971,84.067742
